In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
input_dir = '/kaggle/input'
tsvfiles = []
for dirname, _, filenames in os.walk(input_dir):
    for filename in filenames:
        if filename.endswith('.tsv'):
            file_path = os.path.join(dirname, filename)
            print(file_path)
            tsvfiles.append(file_path)

print(tsvfiles)

/kaggle/input/moodtheme-1/raw_30s_cleantags.tsv
/kaggle/input/moodtheme-1/raw_30s - raw_30s.tsv
/kaggle/input/moodtheme-1/final_mood_labels.tsv
['/kaggle/input/moodtheme-1/raw_30s_cleantags.tsv', '/kaggle/input/moodtheme-1/raw_30s - raw_30s.tsv', '/kaggle/input/moodtheme-1/final_mood_labels.tsv']


In [3]:
tsv_raw = '/kaggle/input/moodtheme-1/raw_30s - raw_30s.tsv'
tsv_raw_Clean = '/kaggle/input/moodtheme-1/raw_30s_cleantags.tsv'
df_raw = pd.read_csv(tsv_raw, delimiter = '\t')
df_raw_cleantags = pd.read_csv(tsv_raw_Clean, delimiter = '\t')

/tmp/ipykernel_24/2283611258.py:3: DtypeWarning: Columns (22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(tsv_raw, delimiter = '\t')
/tmp/ipykernel_24/2283611258.py:4: DtypeWarning: Columns (22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw_cleantags = pd.read_csv(tsv_raw_Clean, delimiter = '\t')


In [4]:
print("Original raw df shape:", df_raw.shape)
print("Original raw df cleantags shape:", df_raw_cleantags.shape)

Original raw df shape: (55701, 50)
Original raw df cleantags shape: (55701, 50)


In [5]:
mismatch = df_raw != df_raw_cleantags
mismatched_cols = mismatch.any(axis=0)
columns_with_mismatches = mismatched_cols[mismatched_cols].index.tolist()

print("Columns with mismatches:", columns_with_mismatches)

Columns with mismatches: ['TAGS', 'TAGS-1', 'TAGS-2', 'TAGS-3', 'TAGS-4', 'TAGS-5', 'TAGS-6', 'TAGS-7', 'TAGS-8', 'TAGS-9', 'TAGS-10', 'TAGS-11', 'TAGS-12', 'TAGS-13', 'TAGS-14', 'TAGS-15', 'TAGS-16', 'TAGS-17', 'TAGS-18', 'TAGS-19', 'TAGS-20', 'TAGS-21', 'TAGS-22', 'TAGS-23', 'TAGS-24', 'TAGS-25', 'TAGS-26', 'TAGS-27', 'TAGS-28', 'TAGS-29', 'TAGS-30', 'TAGS-31', 'TAGS-32', 'TAGS-33', 'TAGS-34', 'TAGS-35', 'TAGS-36', 'TAGS-37', 'TAGS-38', 'TAGS-39', 'TAGS-40', 'TAGS-41', 'TAGS-42', 'TAGS-43', 'TAGS-44']


In [6]:
print(df_raw[df_raw.PATH=='94/7394.mp3']['TAGS-17'])

629    mood/theme---melancolic
Name: TAGS-17, dtype: object


In [7]:
def filter_tagged_musical_instruments(df):
    all_tags = df.iloc[:,5:]
    contains_instrument = all_tags.apply(lambda x: x.astype(str).str.contains('instrument---', na=False)).any(axis=1)
    contains_mood = all_tags.apply(lambda x: x.astype(str).str.contains('mood/theme---', na=False)).any(axis=1)
    # Step 3: Combine conditions to filter rows where BOTH substrings are found
    mask = contains_instrument & contains_mood
    # Step 4: Filter the dataframe using the mask
    filtered_df = df[mask]
    contains_novoice = ~filtered_df.apply(lambda x: x.astype(str).str.contains('instrument---voice', na=False)).any(axis=1)
    novoice_instr_mood = filtered_df[contains_novoice]
    return novoice_instr_mood

In [8]:
df_raw_filtered = filter_tagged_musical_instruments(df_raw)
df_raw_cleantags_filtered = filter_tagged_musical_instruments(df_raw_cleantags)
print("Filtered Raw shape:", df_raw_filtered.shape)
print("Filtered Raw clean tags shape:", df_raw_cleantags_filtered.shape)

Filtered Raw shape: (11592, 50)
Filtered Raw clean tags shape: (11592, 50)


In [9]:
#rows_df_raw.iloc[0]==rows_df_raw_cleantags.iloc[0]
print(df_raw_filtered[df_raw_filtered.PATH=='94/7394.mp3']['TAGS-17'])

629    mood/theme---melancolic
Name: TAGS-17, dtype: object


In [10]:
def filter_unknown(df):
    all_tags = df.iloc[:,5:]
    has_unknown = all_tags.apply(lambda x: x.astype(str).str.contains('unknown', na=False)).any(axis=1)
    df_ = df[has_unknown]
    return df_

unknow_df = filter_unknown(df_raw_filtered)
print(unknow_df)

Empty DataFrame
Columns: [TRACK_ID, ARTIST_ID, ALBUM_ID, PATH, DURATION, TAGS, TAGS-1, TAGS-2, TAGS-3, TAGS-4, TAGS-5, TAGS-6, TAGS-7, TAGS-8, TAGS-9, TAGS-10, TAGS-11, TAGS-12, TAGS-13, TAGS-14, TAGS-15, TAGS-16, TAGS-17, TAGS-18, TAGS-19, TAGS-20, TAGS-21, TAGS-22, TAGS-23, TAGS-24, TAGS-25, TAGS-26, TAGS-27, TAGS-28, TAGS-29, TAGS-30, TAGS-31, TAGS-32, TAGS-33, TAGS-34, TAGS-35, TAGS-36, TAGS-37, TAGS-38, TAGS-39, TAGS-40, TAGS-41, TAGS-42, TAGS-43, TAGS-44]
Index: []

[0 rows x 50 columns]


In [11]:
df_raw_filtered = df_raw_filtered.fillna('NaN')
df_raw_cleantags_filtered = df_raw_cleantags_filtered.fillna('NaN')

In [12]:
print(df_raw_filtered[df_raw_filtered.PATH=='94/7394.mp3']['TAGS-17'])

629    mood/theme---melancolic
Name: TAGS-17, dtype: object


In [13]:
filtered_mismatches = df_raw_filtered!=df_raw_cleantags_filtered
row_mismatches = filtered_mismatches.any(axis=1)
rows_df_raw = df_raw_filtered[row_mismatches]
rows_df_raw_cleantags = df_raw_cleantags_filtered[row_mismatches]

In [14]:
print(rows_df_raw.iloc[0]['TAGS-5'])
print(rows_df_raw_cleantags.iloc[0]['TAGS-5'])

mood/theme---emotion
mood/theme---emotional


In [15]:
print(df_raw_filtered[df_raw_filtered.PATH=='94/7394.mp3']['TAGS-17'])

629    mood/theme---melancolic
Name: TAGS-17, dtype: object


In [16]:
# Define the prefix
def generate_unique_moods(df):
        
    mood_prefix = 'mood/theme---'
    
    # Flatten the DataFrame and filter for values starting with the mood_prefix
    mood_values = df.values.flatten()  # Convert DataFrame to a 1D array
    filtered_mood_values = [value.split('-')[3] for value in mood_values if isinstance(value, str) and value.startswith(mood_prefix)]
    
    # Get the unique mood values
    unique_mood_values = set(filtered_mood_values)
    
    # Print the results
    #print("Unique mood prefixes:", unique_mood_values)
    #print("Number of unique mood prefixes:", len(unique_mood_values))
    return list(unique_mood_values)

In [17]:
raw_unique_moods = generate_unique_moods(df_raw_filtered)
raw_cleantags_unique = generate_unique_moods(df_raw_cleantags_filtered)

In [18]:
print(df_raw_filtered[df_raw_filtered.PATH=='94/7394.mp3']['TAGS-17'])

629    mood/theme---melancolic
Name: TAGS-17, dtype: object


In [19]:
raw_unique_moods

['kids',
 'cheerful',
 'sadness',
 'wedding',
 'video',
 'life',
 'child',
 'surf',
 'oldschool',
 'melancholic',
 'hope',
 'discontent',
 'xmas',
 'chaos',
 'trailermusic',
 'groovy',
 'lovesong',
 'relaxingmusic',
 'reflective',
 'nostalgic',
 'france',
 'tribe',
 'zen',
 'joyful',
 'epicmusic',
 'dynamic',
 'silentfilm',
 'beautiful',
 'future',
 'background',
 'bouncy',
 'cosmic',
 'relaxing',
 'education',
 'slow',
 'energy',
 'youtube',
 'suspense',
 'mexico',
 'sad',
 'action',
 'mystery',
 'africa',
 'videogame',
 'urban',
 'meditative',
 'india',
 'filmmusic',
 'melancolic',
 'fast',
 'spiritual',
 'scary',
 'western',
 'tv',
 'science',
 'ballad',
 'laidback',
 'sex',
 'poetry',
 'darkness',
 'horror',
 'motivational',
 'adult',
 'catchy',
 'driving',
 'emotional',
 'sciencefiction',
 'travel',
 'robot',
 'festive',
 'hopeful',
 'fantasy',
 'chilled',
 'soundscape',
 'traditional',
 'uplifting',
 'summer',
 'cool',
 'success',
 'motivation',
 'argentina',
 'irish',
 'rain',
 

In [20]:
#implementing word2vec
!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 39.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.3 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is incompatible.
tsfresh 0.20.3 requires scipy>=1

In [21]:
import numpy as np
import gensim
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
# Function to load GloVe embeddings
def load_glove_embeddings(file_path):
    glove_model = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            glove_model[word] = vector
    return glove_model

# Example path for GloVe file (adjust accordingly)
glove_file_path = '/kaggle/input/glove6b100d/glove.6B.100d.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)

# Define the 4 emotions and example tags
#emotions = ['happy', 'sad', 'calm', 'energetic']
emotions = ['happy', 'calm', 'tense', 'energetic']


# Example function to get vector for a word
def get_vector(word, glove_model):
    return glove_model.get(word)

# Compute similarity between tags and emotions
def classify_tags(raw_unique_moods, emotions, glove_model):
    classified_tags = {emotion : [] for emotion in emotions}
    notfound_tags = []
    
    # Get emotion vectors
    emotion_vectors = {emotion: get_vector(emotion, glove_model) for emotion in emotions}
    
    # Classify each tag
    for tag in raw_unique_moods:
        tag_vector = get_vector(tag, glove_model)
        #print(f"Tag {tag}")
        if tag_vector is not None:
            similarities = {emotion: cosine_similarity([tag_vector], [emotion_vectors[emotion]])[0][0] for emotion in emotions}
            #print(f'Tag {tag} similarities: {similarities}')
            best_emotion = max(similarities, key=similarities.get)
            #print("Best emotion:", best_emotion)
            classified_tags[best_emotion].append(tag)
        else:
            print(f"Tag '{tag}' not found in GloVe embeddings.")
            notfound_tags.append(tag)
    
    return classified_tags, notfound_tags

# Classify tags
classified_tags, unclassified_tags = classify_tags(raw_unique_moods, emotions, glove_embeddings)

Tag 'oldschool' not found in GloVe embeddings.
Tag 'trailermusic' not found in GloVe embeddings.
Tag 'relaxingmusic' not found in GloVe embeddings.
Tag 'epicmusic' not found in GloVe embeddings.
Tag 'silentfilm' not found in GloVe embeddings.
Tag 'filmmusic' not found in GloVe embeddings.
Tag 'melancolic' not found in GloVe embeddings.
Tag 'sciencefiction' not found in GloVe embeddings.
Tag 'energic' not found in GloVe embeddings.
Tag 'festif' not found in GloVe embeddings.
Tag 'filmscore' not found in GloVe embeddings.
Tag 'folkrock' not found in GloVe embeddings.
Tag 'planant' not found in GloVe embeddings.
Tag 'lovemusic' not found in GloVe embeddings.


In [23]:
# Print classified tags
for emotion, tags_list in classified_tags.items():
    print(f"{emotion.capitalize()}: {tags_list}\n")

Happy: ['kids', 'wedding', 'video', 'life', 'child', 'surf', 'hope', 'xmas', 'groovy', 'nostalgic', 'france', 'tribe', 'beautiful', 'future', 'education', 'slow', 'youtube', 'mexico', 'sad', 'action', 'mystery', 'africa', 'fast', 'scary', 'tv', 'science', 'ballad', 'sex', 'horror', 'driving', 'emotional', 'travel', 'hopeful', 'fantasy', 'traditional', 'summer', 'success', 'motivation', 'irish', 'angry', 'fun', 'happiness', 'party', 'drive', 'drunk', 'movie', 'beach', 'dancing', 'nature', 'romantic', 'film', 'loud', 'documentary', 'love', 'drama', 'history', 'news', 'vintage', 'lazy', 'sweet', 'tech', 'optimistic', 'silly', 'lullaby', 'psycho', 'dark', 'comedy', 'advertising', 'health', 'adventure', 'holiday', 'depressed', 'children', 'dream', 'soft', 'winter', 'happy', 'excited', 'nice', 'sport', 'sleepy', 'thankful', 'halloween', 'anxious', 'strange', 'funny', 'brazil', 'berlin', 'japan', 'celebration', 'game', 'corporate', 'commercial', 'romance', 'computers', 'fashion', 'joy', 'craz

In [ ]:
print(df_raw_filtered[df_raw_filtered.PATH=='94/7394.mp3']['TAGS-17'])

In [24]:
print("Length of unclassified tags:", len(unclassified_tags))

Length of unclassified tags: 14


In [26]:
map_unclassified = {'silentfilm': 'silent',
                    'melancolic':'melancholic',
                    'energic':'energetic',
                    'festif': 'festive',
                    'epicmusic':'epic',
                    'lovemusic':'love',
                    'sciencefiction':'scifi',
                    'trailermusic':'trailer',
                    'relaxingmusic':'relaxing',
                    'planant':'spacy',
                    'oldschool':'retro',
                    'filmmusic':'film',
                    'folkrock': 'folk',
                    'filmscore': 'film'
                   }

updated_words_list = [map_unclassified.get(word, word) for word in raw_unique_moods]

print(updated_words_list)

['kids', 'cheerful', 'sadness', 'wedding', 'video', 'life', 'child', 'surf', 'retro', 'melancholic', 'hope', 'discontent', 'xmas', 'chaos', 'trailer', 'groovy', 'lovesong', 'relaxing', 'reflective', 'nostalgic', 'france', 'tribe', 'zen', 'joyful', 'epic', 'dynamic', 'silent', 'beautiful', 'future', 'background', 'bouncy', 'cosmic', 'relaxing', 'education', 'slow', 'energy', 'youtube', 'suspense', 'mexico', 'sad', 'action', 'mystery', 'africa', 'videogame', 'urban', 'meditative', 'india', 'film', 'melancholic', 'fast', 'spiritual', 'scary', 'western', 'tv', 'science', 'ballad', 'laidback', 'sex', 'poetry', 'darkness', 'horror', 'motivational', 'adult', 'catchy', 'driving', 'emotional', 'scifi', 'travel', 'robot', 'festive', 'hopeful', 'fantasy', 'chilled', 'soundscape', 'traditional', 'uplifting', 'summer', 'cool', 'success', 'motivation', 'argentina', 'irish', 'rain', 'angry', 'sexy', 'ethereal', 'fun', 'happiness', 'silence', 'arabic', 'spacey', 'heavy', 'energetic', 'warm', 'powerful

In [27]:
print('trailermusic' in updated_words_list)
print('trailermusic' in raw_unique_moods)

False
True


In [28]:
new_classified_tags, new_unclassified_tags = classify_tags(updated_words_list, emotions, glove_embeddings)

In [29]:
print(len(new_unclassified_tags))

0


In [30]:
print(df_raw_filtered[df_raw_filtered.PATH=='94/7394.mp3']['TAGS-17'])

629    mood/theme---melancolic
Name: TAGS-17, dtype: object


In [31]:
# Print classified tags
for emotion, tags_list in new_classified_tags.items():
    print(f"{emotion.capitalize()}: {tags_list}\n")

Happy: ['kids', 'wedding', 'video', 'life', 'child', 'surf', 'hope', 'xmas', 'trailer', 'groovy', 'nostalgic', 'france', 'tribe', 'silent', 'beautiful', 'future', 'education', 'slow', 'youtube', 'mexico', 'sad', 'action', 'mystery', 'africa', 'film', 'fast', 'scary', 'tv', 'science', 'ballad', 'sex', 'horror', 'driving', 'emotional', 'travel', 'hopeful', 'fantasy', 'traditional', 'summer', 'success', 'motivation', 'irish', 'angry', 'fun', 'happiness', 'party', 'drive', 'drunk', 'movie', 'beach', 'dancing', 'nature', 'romantic', 'film', 'loud', 'documentary', 'love', 'drama', 'history', 'news', 'vintage', 'lazy', 'sweet', 'tech', 'optimistic', 'silly', 'lullaby', 'psycho', 'dark', 'film', 'comedy', 'advertising', 'health', 'adventure', 'holiday', 'depressed', 'folk', 'children', 'dream', 'soft', 'winter', 'happy', 'excited', 'nice', 'sport', 'sleepy', 'thankful', 'halloween', 'anxious', 'strange', 'funny', 'brazil', 'berlin', 'japan', 'celebration', 'game', 'corporate', 'commercial', 'r

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
category_dict = new_classified_tags
label_to_category = {label: category for category, labels in category_dict.items() for label in labels}

In [ ]:
print(label_to_category['trailer'])

In [ ]:
print(df_raw_filtered[df_raw_filtered.PATH=='94/7394.mp3']['TAGS-17'])
print(df_raw_filtered[df_raw_filtered.PATH=='48/948.mp3']['TAGS-6'])

In [ ]:
df_raw_filtered = df_raw_filtered.astype(str)

In [ ]:
subset_df = df_raw_filtered.copy()

In [ ]:
# Function to replace mood labels with broader categories
mood_prefix = 'mood/theme---'
def replace_mood_labels(cell_value):
    #print(cell_value)
    check_cell = cell_value
    if isinstance(cell_value, pd.Series):
        check_cell = cell_value.tolist()[0]
        
    if isinstance(cell_value, object) and check_cell.startswith(mood_prefix):
        mood_label = check_cell.split(mood_prefix)[1]
        if mood_label in map_unclassified:
            mood_label = map_unclassified[mood_label]
            #DEBUG print(f"unclassified: {check_cell} with fixed mood label {mood_label}")
          # Extract mood label
        category_label = label_to_category.get(mood_label)
        #DEBUG print(check_cell, category_label)
    # If not found in label_to_category, check the mood_replacement_dict
        if category_label is None:
            category_label = map_unclassified.get(mood_label, 'unknown')  # Default to 'unknown' if not found in either

        #DEBUG print(f"{check_cell} {category_label}")
        return f"{mood_prefix}{category_label}"  # Replace with broader category or 'unknown'
    return cell_value  # Return cell as is if no change is needed


In [ ]:
#subset_df10 = subset_df.iloc[:100]
#subset_df10.shape
#for col in subset_df10.columns:
#sample_df = subset_df[subset_df.PATH=='94/7394.mp3'].apply(replace_mood_labels)
sample_df = subset_df[subset_df.PATH=='94/7394.mp3'].apply(lambda x: replace_mood_labels(x))

In [ ]:
print(sample_df[subset_df.PATH=='94/7394.mp3']['TAGS-17'])

In [ ]:
# Apply replacement to columns with 'mood---' prefix
def update_df_new_moods(dfx):
    for col in dfx.columns:
        #if df[col].str.startswith(mood_prefix).any():  # Check if column contains mood labels
        dfx[col] = dfx[col].apply(lambda x: replace_mood_labels(x))
    return dfx

# Output the updated DataFrame
modified_raw_moods = update_df_new_moods(subset_df)
#print(modified_raw_moods)

In [ ]:
print(modified_raw_moods[modified_raw_moods.PATH=='94/7394.mp3']['TAGS-17'])
print(df_raw_filtered[df_raw_filtered.PATH=='94/7394.mp3']['TAGS-17'])

In [ ]:
modified_raw_moods

In [ ]:
def get_cell_indices(dfx):
    matching_indices = []  # List to store matching indices

    def check_cell_value(row):
        # Loop through each column in the row
        for col in dfx.columns:
            cell_value = row[col]
            if isinstance(cell_value, str) and cell_value.startswith('mood/theme---'):
                # Append the tuple (row index, column index) to the list
                matching_indices.append((row.name, col, cell_value))

    # Apply the function to each row
    dfx.apply(check_cell_value, axis=1)

    return matching_indices


In [ ]:
indices = get_cell_indices(modified_raw_moods)
moods = [mood[2] for mood in indices]

print("Matching moods:", set(moods))

In [ ]:
# Example list of tuples
# Create the dictionary
result_dict = {}
for t in indices:
    key = t[2]  # Third element as the dictionary key
    value = (t[0], t[1])  # First two elements as the tuple value
    if key not in result_dict:
        result_dict[key] = []  # Initialize a list if the key doesn't exist
    result_dict[key].append(value)  # Append the value tuple to the key's list

# Print the resulting dictionary
#print(result_dict)


In [ ]:
result_dict.keys()

In [ ]:
###DEBUG PURPOSE ONLY, RUN IFF ABOVE HAS >4 LABELS
for index, row in modified_raw_moods.iterrows():
    if row.name == 21375:  # Example: Check for a specific row.name
        # Extract the row using loc
        extracted_row = modified_raw_moods.loc[row.name]
        print(f"Extracted Row:\n{extracted_row}")

In [ ]:
result_dict['mood/theme---epic']

In [ ]:
def extract_final_mood(row):
    # Select all columns that may contain mood labels
    #print("Row:", row)
    mood_columns = [col for col in row.index if row[col] is not None and 'mood/theme---' in str(row[col])]
    #print("Mood columns:", mood_columns)
    if mood_columns:
        # Take the first mood label encountered
        first_mood = row[mood_columns[0]].split('mood/theme---')[1]
        #print("first_mood:", first_mood)
        return first_mood
    else:
        # If no mood label exists, return None
        return None

# Apply the function to create the final mood label
df2 = modified_raw_moods.copy()
df2['final_mood'] = df2.apply(extract_final_mood, axis=1)

# Select only the 'PATH' and 'final_mood' columns
final_df = df2[['PATH', 'final_mood']]

# Print the resulting DataFrame
print(final_df)

In [ ]:
final_df['final_mood'].value_counts()

In [ ]:
keyword = 'unknown'
result = final_df[final_df['final_mood'].str.contains(keyword, case=False, na=False)]
print(result)

In [ ]:
final_df.to_csv('/kaggle/working/final_mood_labels.tsv', sep='\t', index=False)